# Import module

In [3]:
import pandas as pd
import jaconv
from bs4 import BeautifulSoup 
from urllib import request
from urllib.error import HTTPError
import matplotlib.pyplot as plt
import numpy as np
import pickle
import datetime

# Change fontype of pyplot
import matplotlib.font_manager
#print([f.name for f in matplotlib.font_manager.fontManager.ttflist])
plt.rcParams['font.family'] = 'MS Gothic'
# For mac use
#plt.rcParams['font.family'] = 'AppleGothic'

# If you want to downlod the most recent data then you should not skip this.
skip = 0

# Define urls

In [4]:
# url headers
url_head = "https://www5.cao.go.jp/keizai3/"
url_new = "https://www5.cao.go.jp/keizai3/watcher_index.html"
url_old = "https://www5.cao.go.jp/keizai3/kako_watcher.html"
url_old_2 = "https://www5.cao.go.jp/keizai3/kako_csv/kako2_watcher.html"
# 
kigou2value = {"◎":5,"○":4,"□":3,"▲":2,"×":1}

# Define functions

In [5]:
def get_all_url(target):
    '''
    Description:webページ内の全urlを取得する関数
    入力：各調査が一覧になったページ
    出力：各調査のcsvがあるurl一覧
    '''
    data = request.urlopen(target).read()
    soup = BeautifulSoup(data, 'html.parser')
    tmp_links = soup.find_all("a")
    link = []
    for a in tmp_links:
        try:
            if "/menu.html" in a.attrs['href']:
                
                if a.attrs['href'][0:4] < str(2010):
                    #print(a.attrs['href'][0:4])
                    pass
                else:
                    link.append(a.attrs['href'])
            elif ".csv" in a.attrs['href']:
                link.append(a.attrs['href'])
        except:
            pass
    return link


def extract_data(df_in):
    '''
    Description:各調査のcsvがあるurlからデータフレームを直接作る関数
    入力：各調査のcsvに直接アクセスできるurl
    出力：データフレーム
    '''  
    han_numbers = [str(_) for _ in range(10)]
    kigou_list = ["◎","○","□","▲","×"]
    region_list = ["北海道","東北","北関東","南関東","甲信越",
                   "東海","北陸","近畿","中国","四国","九州","沖縄"]
    
    place = 6
    sakiyuki = 0
    for l in range(len(df_in)):
        if ('景気の現状判断' in str(df_in.iloc[l,:])):
            place = l
            sakiyuki = 0
            break
            
        if ("景気の先行き判断" in str(df_in.iloc[l,:])):
            place = l
            sakiyuki = 1
            break
    
    col_names = df_in.iloc[place,:]
    use_col_names = dict()
    for l in range(len(col_names)):
        use_col_names.update({col_names[l]:l})
        
    out_data = []
    
    # Used to determine which regions
    region = ""

    # Loop through all records
    for i in range(len(df_in)):
        line = df_in.iloc[i,0]

        # Update line headers
        if line == line:
            new_region = 0
            for r in region_list:
                if r in line:
                    new_region = 1
                    break
                   
            if new_region == 1:
                region = r
        
        line = df_in.iloc[i,2]
        if line == line:
            for kigou in kigou_list:
                if kigou in line:
                    if sakiyuki == 0:
                    
                        if "判断の理由" in use_col_names:

                            out_data.append([region,
                                             df_in.iloc[i,use_col_names["景気の現状判断"]],
                                             df_in.iloc[i,use_col_names["業種・職種"]],
                                             df_in.iloc[i,use_col_names["判断の理由"]],
                                             df_in.iloc[i,use_col_names["追加説明及び具体的状況の説明"]]])
                        else:
                            out_data.append([region,
                                             df_in.iloc[i,use_col_names["景気の現状判断"]],
                                             df_in.iloc[i,use_col_names["業種・職種"]],
                                             "",
                                             df_in.iloc[i,use_col_names["追加説明及び具体的状況の説明"]]])
                            
                    else:
                        if "判断の理由" in use_col_names:

                            out_data.append([region,
                                             df_in.iloc[i,use_col_names["景気の先行き判断"]],
                                             df_in.iloc[i,use_col_names["業種・職種"]],
                                             df_in.iloc[i,use_col_names["判断の理由"]],
                                             df_in.iloc[i,use_col_names["景気の先行きに対する判断理由"]]])
                        else:
                            out_data.append([region,df_in.iloc[i,use_col_names["景気の先行き判断"]],
                                             df_in.iloc[i,use_col_names["業種・職種"]],
                                             "",
                                             df_in.iloc[i,use_col_names["景気の先行きに対する判断理由"]]])     
                        
    df_out = pd.DataFrame(out_data)
    df_out.columns = ["region","symbol","industry","topic","text"]
    
    return(df_out)

# Get urls

In [6]:
# retrieve all urls
all_urls = get_all_url(url_new)
all_urls.extend(get_all_url(url_old))
all_urls.extend(get_all_url(url_old_2))
all_urls.sort()

In [7]:
all_urls

['2010/0112watcher/menu.html',
 '2010/0208watcher/menu.html',
 '2010/0308watcher/menu.html',
 '2010/0408watcher/menu.html',
 '2010/0513watcher/menu.html',
 '2010/0608watcher/menu.html',
 '2010/0708watcher/menu.html',
 '2010/0809watcher/menu.html',
 '2010/0908watcher/menu.html',
 '2010/1008watcher/menu.html',
 '2010/1109watcher/menu.html',
 '2010/1208watcher/menu.html',
 '2011/0112watcher/menu.html',
 '2011/0208watcher/menu.html',
 '2011/0308watcher/menu.html',
 '2011/0408watcher/menu.html',
 '2011/0512watcher/menu.html',
 '2011/0608watcher/menu.html',
 '2011/0708watcher/menu.html',
 '2011/0808watcher/menu.html',
 '2011/0908watcher/menu.html',
 '2011/1011watcher/menu.html',
 '2011/1109watcher/menu.html',
 '2011/1208watcher/menu.html',
 '2012/0112watcher/menu.html',
 '2012/0208watcher/menu.html',
 '2012/0308watcher/menu.html',
 '2012/0409watcher/menu.html',
 '2012/0510watcher/menu.html',
 '2012/0608watcher/menu.html',
 '2012/0709watcher/menu.html',
 '2012/0808watcher/menu.html',
 '2012/0

# Get all data

In [8]:
%%time
if skip != 1:
    error_urls, list_dfs = [],[]
    #for i in range(-200,-201):
    for i in range(len(all_urls)):
        print(i)
        if ".csv" in all_urls[i]:
            date = str(1988 + int(all_urls[i][1:3])) +"_" +  str(all_urls[i][3:5]) + "_" +  "01"
            file_url = "https://www5.cao.go.jp/keizai3/kako_csv/" +  all_urls[i]
            try:
                df_tmp = pd.read_csv(file_url,encoding="cp932")
                df_out = extract_data(df_tmp)
                df_out["date"] = date
                if "watcher4" in all_urls[i]:
                    df_out["type"] = "now"
                else:
                    df_out["type"] = "future"
                list_dfs.append(df_out)

            except:
                error_urls.append(file_url)
    
        else:
        
            date = all_urls[i][0:4] +"_" +  all_urls[i][5:7] + "_" +  all_urls[i][7:9] 
            # 現状
            file_url = url_head + all_urls[i].strip("menu.html") + "watcher4.csv"
            try:
                df_tmp = pd.read_csv(file_url,encoding="cp932")
                df_out = extract_data(df_tmp)
                df_out["date"] = date
                df_out["type"] = "now"
                list_dfs.append(df_out)

            except:
                error_urls.append(file_url)
               
            


            # 先行き
            file_url = url_head + all_urls[i].strip("menu.html") + "watcher5.csv"

            try:
                df_tmp = pd.read_csv(file_url,encoding="cp932")
                df_out = extract_data(df_tmp)
                df_out["date"] = date
                df_out["type"] = "future"
                list_dfs.append(df_out)

            except:
                #print(file_url)
                error_urls.append(file_url)

            # 甲信越　現状
            file_url = url_head + all_urls[i].strip("menu.html") + "watcher6.csv"
            try:
                df_tmp = pd.read_csv(file_url,encoding="cp932")
                df_out = extract_data(df_tmp)
                df_out["date"] = date
                df_out["type"] = "now_koushin"
                list_dfs.append(df_out)

            except:
                error_urls.append(file_url)

            # 甲信越　先行き
            file_url = url_head + all_urls[i].strip("menu.html") + "watcher7.csv"
            try:
                df_tmp = pd.read_csv(file_url,encoding="cp932")
                df_out = extract_data(df_tmp)
                df_out["date"] = date
                df_out["type"] = "future_koushin"
                list_dfs.append(df_out)

            except:
                error_urls.append(file_url)

0
1


/var/folders/xx/46bbfzhs77j5z84b92fz834w0000gn/T/ipykernel_8896/3658594269.py:54: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  use_col_names.update({col_names[l]:l})
/var/folders/xx/46bbfzhs77j5z84b92fz834w0000gn/T/ipykernel_8896/3658594269.py:54: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  use_col_names.update({col_names[l]:l})
/var/folders/xx/46bbfzhs77j5z84b92fz834w0000gn/T/ipykernel_8896/3658594269.py:54: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by positi

2


/var/folders/xx/46bbfzhs77j5z84b92fz834w0000gn/T/ipykernel_8896/3658594269.py:54: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  use_col_names.update({col_names[l]:l})
/var/folders/xx/46bbfzhs77j5z84b92fz834w0000gn/T/ipykernel_8896/3658594269.py:54: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  use_col_names.update({col_names[l]:l})
/var/folders/xx/46bbfzhs77j5z84b92fz834w0000gn/T/ipykernel_8896/3658594269.py:54: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by positi

3


...

414
415


/var/folders/xx/46bbfzhs77j5z84b92fz834w0000gn/T/ipykernel_8896/3658594269.py:54: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  use_col_names.update({col_names[l]:l})
/var/folders/xx/46bbfzhs77j5z84b92fz834w0000gn/T/ipykernel_8896/3658594269.py:54: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  use_col_names.update({col_names[l]:l})


416
417


/var/folders/xx/46bbfzhs77j5z84b92fz834w0000gn/T/ipykernel_8896/3658594269.py:54: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  use_col_names.update({col_names[l]:l})
/var/folders/xx/46bbfzhs77j5z84b92fz834w0000gn/T/ipykernel_8896/3658594269.py:54: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  use_col_names.update({col_names[l]:l})


418
CPU times: user 39 s, sys: 1.99 s, total: 40.9 s
Wall time: 2min 23s


# Concatenate all together

In [9]:
if skip != 1:
    # concatenate
    df = pd.concat(list_dfs)

    # change symbols to values
    df["label"] = df["symbol"].map(kigou2value)

    # count type
    df["type"].value_counts()

# Save

In [10]:
if skip != 1:
    # save
    df.to_csv("SuperD_Class5_keiki.csv",index=False)

# Load

In [11]:
df["date"].value_counts()

date
2020_04_08    2948
2020_05_13    2941
2020_06_08    2898
2014_04_08    2891
2013_09_09    2877
              ... 
2000_05_01     220
2000_04_01     203
2000_02_01     201
2000_03_01     190
2000_01_01     145
Name: count, Length: 290, dtype: int64

In [12]:
# Load
df = pd.read_csv("SuperD_Class5_keiki.csv",low_memory=False)

# change to datetime format
df["date"] = pd.to_datetime(df["date"],format="%Y_%m_%d")

In [13]:
df

,region,symbol,industry,topic,text,date,type,label
0,北海道,◎,－,－,－,2010-02-08,now,5.0
1,北海道,○,商店街（代表者）,販売量の動き,・例年にないほど寒く、雪も多かったことから、防寒靴や手袋・マフラーなどの軽衣料などが良く売れ...,2010-02-08,now,4.0
2,北海道,○,百貨店（販売促進担当）,来客数の動き,・年末から来客数が微増で推移している。,2010-02-08,now,4.0
3,北海道,○,スーパー（店長）,販売量の動き,・３か月前と比較して販売量は減っているが、日曜が１日多かったこともあり、前年比は104.7％...,2010-02-08,now,4.0
4,北海道,○,旅行代理店（従業員）,来客数の動き,・１月半ばから来客数が増えてきている。ただ、低価格志向は相変わらずであり、売上が一気に好転す...,2010-02-08,now,4.0
...,...,...,...,...,...,...,...,...
656074,沖縄,○,＊,NaN,＊,2009-12-01,future,4.0
656075,沖縄,□,求人情報誌製作会社（総務担当）,NaN,・まだまだ景気回復の兆しが見られず、当面この状況が続く。,2009-12-01,future,3.0
656076,沖縄,▲,学校［専門学校］（就職担当）,NaN,・県内外のほとんどの企業の求人数が減少している。理由として次年度の景気の先行きが「不透明」だ...,2009-12-01,future,2.0
656077,沖縄,▲,学校［大学］（就職担当）,NaN,・県内企業の、採用を抑える方針は今後も続くと見込まれる。,2009-12-01,future,2.0
